In [1]:
%matplotlib inline
import supervision as sv
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from ultralytics import YOLO
import plotly.express as px
import pandas as pd
import numpy as np
import torch
import cv2
import os

In [2]:
HOME = os.getcwd()
print(f"{HOME = }")
print(f"{torch.cuda.is_available() = }")

HOME = 'c:\\Repos\\inz'
torch.cuda.is_available() = True


In [3]:
model = YOLO('yolov8n.pt')  # Load a pretrained YOLO model https://docs.ultralytics.com/usage/python/

In [4]:
video_path = r'C:\Repos\yolov5\data\vids\hway.mkv'
target_path = r'C:\Repos\inz\hway_test.mkv'

In [5]:
classes = {1:'bicycle', 2:'car', 3:'motorcycle', 5:'bus', 7:'truck'}

In [6]:
# video_info = VideoInfo.from_video_path(video_path)
# print("TUTAJ: ", video_info)
# cap = cv2.VideoCapture(video_path)
# out = cv2.VideoWriter(target_path, cv2.VideoWriter_fourcc(*'DIVX'), video_info.fps, video_info.resolution_wh)
# frame_no = 0
# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         cv2.destroyAllWindows()
#         break

#     # results = model(source=frame[440:,:], classes=[1, 2, 3, 5, 7], imgsz=640, conf=0.25, show=False)[0]  # (source=frame[440:,:]
#     # detections = []
#     # for r in results.boxes.data.tolist():
#     #     x1, y1, x2, y2, score, class_id = r
#     #     detections.append([int(x1 + 0.5), int(y1 + 0.5), int(x2 + 0.5), int(y2 + 0.5), score, int(class_id)])

#     # # display current frame
#     # cv2.imshow(f"frame {0}", frame)
#     # if cv2.waitKey(30) & 0xFF == 27:
#     #     cv2.destroyAllWindows()
#     #     break
    
#     frame_no += 1
#     out.write(frame)

# cap.release()
# out.release()

In [7]:
# video_info = sv.VideoInfo.from_video_path(video_path)
# with sv.VideoSink(target_path, video_info) as s:
#     for frame in tqdm(sv.get_video_frames_generator(video_path), total=video_info.total_frames-2):
#         result = model(source=frame, classes=list(classes.keys()), imgsz=640, conf=0.25, show=False)[0]  # (source=frame[440:,:]

#         # detections = []
#         # for r in results.boxes.data.tolist():
#         #     x1, y1, x2, y2, score, class_id = r
#         #     detections.append([int(x1 + 0.5), int(y1 + 0.5), int(x2 + 0.5), int(y2 + 0.5), score, int(class_id)])
#         # for d in detections:
#         #     # if (d[1]+d[3])/2 < 440:
#         #     cv2.rectangle(frame, (d[0],d[1]), (d[2],d[3]), (0,0,255))

#         detections = sv.Detections.from_yolov8(result)
#         box_annotator = sv.BoxAnnotator(color=sv.ColorPalette.default(),  # sv.ColorPalette.default()
#                                         thickness=2,
#                                         text_color=sv.Color.black(),
#                                         text_scale=0.5,
#                                         text_thickness=1,
#                                         text_padding=5)
#         labels = [
#             f"{classes[class_id]} {confidence:0.2f}"
#             for _, _, confidence, class_id, _ in detections
#         ]
#         annotated_frame = box_annotator.annotate(
#             scene=frame,  # scene=frame.copy()
#             detections=detections,
#             labels=labels
#         )

#         s.write_frame(frame)

In [8]:
# Process Whole Video
results = model.track(source=video_path,
                      stream=False,  # stream=True should be used for long videos to avoid OOM errors
                      tracker='botsort.yaml',  # custombotsort.yaml
                      persist=True,
                      classes=list(classes.keys()),
                      imgsz=640,
                      conf=0.25,
                      iou=0.7,
                      project=r'C:\Repos\inz\runs',
                      name='hway_test',
                      save=True,
                      show=False,
                      line_width=3,
                      # save_txt=True,
                      # save_conf=True,
                      # agnostic_nms=False,
                      # visualize=False,
                      # save_crop=True,
                      )



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/681) C:\Repos\yolov5\data\vids\hway.mkv: 384x640 3 cars, 1 bus, 1 truck, 77.0ms
video 1/1 (2/681) C:\Repos\yolov5\data\vids\hway.mkv: 384x640 3 cars, 1 bus, 1 truck, 4.0ms
video 1/1 (3/681) C:\Repos\yolov5\data\vids\hway.mkv: 384x640 3 cars, 1 bus, 1 truck, 3.0ms
video 1/1 (4/681) C:\Repos\yolov5\data\vids\hway.mkv: 384x640 3 cars, 1 bus, 1 truck, 4.0ms
video 1/1 (5/681) C:\Repos\yolov5\data\vids\hway.mkv: 384x640 3 cars, 1 bus, 1 truck, 3.0ms


In [9]:
temp_table = []
for frame, r in enumerate(results):
    for row in r.boxes.data.cpu().tolist():
        x1, y1, x2, y2, id, conf, class_no = row
        temp_table.append([int(id), int(class_no), conf, frame, x1, y1, x2, y2])
df = pd.DataFrame(temp_table, columns=['id', 'class_no', 'confidence', 'frame', 'x1', 'y1', 'x2', 'y2'])
df
# df[df['id']==1]

,id,class_no,confidence,frame,x1,y1,x2,y2
0,1,2,0.732253,0,793.268921,520.342529,858.281982,576.873779
1,2,7,0.698407,0,206.710098,500.528870,661.444336,930.695984
2,3,2,0.676815,0,636.747681,746.690308,773.570984,873.658203
3,4,2,0.646440,0,1218.395752,623.618591,1345.002563,734.412781
4,5,5,0.623087,0,1140.454590,464.952972,1218.850830,562.236694
...,...,...,...,...,...,...,...,...
5169,59,2,0.488132,678,1232.520752,627.364807,1338.267822,715.926880
5170,60,5,0.813089,678,754.528687,416.822540,834.394958,507.097809
5171,61,2,0.884061,678,687.647339,674.271423,804.491272,773.107788
5172,62,2,0.702702,678,1031.453979,467.628143,1078.552002,509.050262


In [10]:
unique_classes = df['class_no'].unique()
frames = {class_no:[None]*len(unique_classes) for class_no in df['frame'].unique()}
for i, class_no in enumerate(unique_classes):
    series = df[df['class_no']==class_no]['frame'].value_counts()
    for frame, count in zip(series.index, series.values):
        frames[frame][i] = count
df_class_counts = pd.DataFrame.from_dict(frames, orient='index', columns=[classes[n] for n in unique_classes])  # , dtype={c:int for c in unique_classes}
df_class_counts

,car,truck,bus
0,3,1.0,1.0
1,3,1.0,1.0
2,3,1.0,1.0
3,3,1.0,1.0
4,3,1.0,1.0
...,...,...,...
674,5,NaN,1.0
675,6,NaN,1.0
676,6,NaN,1.0
677,6,NaN,1.0


In [11]:
fig = px.line(df_class_counts)
fig.update_layout(title=dict(text="Number of objects detected in each frame"), xaxis_title="#frame", yaxis_title="count")
fig.show()

In [12]:
fig = px.bar(df_class_counts)
fig.update_layout(title=dict(text="Number of objects detected in each frame"), xaxis_title="#frame", yaxis_title="count")
fig.show()

In [13]:
fig = px.histogram(df_class_counts, text_auto=True)
fig.update_layout(
    title=dict(text="Histogram of detections"),
    xaxis_title="#detections",
    yaxis_title="count",
    xaxis=dict(dtick=1),
    bargap=0.1
    )
fig.show()

In [45]:
temp_df = pd.DataFrame(df['class_no'].value_counts())
fig = px.pie(temp_df, values='count', names=[classes[i] for i in temp_df.index])
fig.update_layout(title=dict(text="Total Detections"))
fig.show()